<a href="https://colab.research.google.com/github/jeongjin-shin/CartPole-DQN-Tensorflow/blob/main/cartpole_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
import numpy as np
import gym
import random
from keras.layers import *

In [2]:
model = keras.models.Sequential()

model.add(Dense(12,activation = 'tanh',input_shape = (4,)))
model.add(Dense(2,activation = 'linear')) 

In [4]:
model.compile(optimizer = 'adam',loss = 'MSE',metrics = ['accuracy'])

In [9]:
model= keras.models.load_model("cartpole-dqn490.h5")

In [10]:
env = gym.make("CartPole-v1")

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                60        
                                                                 
 dense_1 (Dense)             (None, 2)                 26        
                                                                 
Total params: 86
Trainable params: 86
Non-trainable params: 0
_________________________________________________________________


In [12]:
def preprocessing(state):
    return np.reshape(state,(1,4))

In [ ]:
replay_mem = []
replay_size = 32
reward_list = []
local_mean = list()
for episode in range(490,2000):

    done = False
    state = env.reset()
    count = 0
    reward_tot = 0
    model.save("cartpole-dqn.h5")

    while not done:
        count += 1
        e = (1/(episode/200+1))
        
        
        #epslion greedy search
        Q = model.predict(preprocessing(state))
        if e>np.random.rand(1):
            action = env.action_space.sample()
        else:
            action = np.argmax(Q)
            
            
        #take_action and set reward
        state_next, reward, done, info = env.step(action)
        if done:
            reward = - 100
            
        
        #replay_mem
        replay_mem.append([state,action,reward,state_next,done])
        
        if len(replay_mem)>2048:
            del replay_mem[0]
            
            
        state = state_next
        reward_tot += reward
        state = state_next
        
        Q_list = []
        state_list = []
        
        
        #set this_replay_Size
        if len(replay_mem)<replay_size:
            this_replay_size = len(replay_mem)
        else:
            this_replay_size = replay_size
            
            
        #sample random batch from replay_memory
        for sample in random.sample(replay_mem,this_replay_size):
            state_m,action_m,reward_m,state_next_m,done_m = sample
            if done:
                Q[0,action] = reward_m
            else:
                Q_new = model.predict(preprocessing(state_next_m))
                Q[0,action] = reward_m + 0.97*np.max(Q_new)
            Q_list.append(Q)
            state_list.append(state_m)
              
        #convert to nupmy array and train
        Q_list = np.array(Q_list)        
        state_list = np.array(state_list)
        
        hist = model.fit(state_list,Q_list,epochs = 5,verbose = 0)
        #print("Done :",done,"     Reward :",reward,"     Reward_total :",reward_tot)
    
    local_mean.append(reward_tot)
    reward_list.append(reward_tot)
    if episode%10 == 0:
        print("Episode :",episode+1,"     Reward_total :", reward_tot,"     Reward_local_mean :",np.mean(local_mean))
        local_mean = list()
        
print("*******End Learning")

Episode : 491      Reward_total : -87.0      Reward_local_mean : -87.0
Episode : 501      Reward_total : -90.0      Reward_local_mean : -89.8
Episode : 511      Reward_total : -91.0      Reward_local_mean : -86.9
Episode : 521      Reward_total : -90.0      Reward_local_mean : -89.6
Episode : 531      Reward_total : -84.0      Reward_local_mean : -82.8
Episode : 541      Reward_total : -89.0      Reward_local_mean : -88.5
Episode : 551      Reward_total : -92.0      Reward_local_mean : -90.0
Episode : 561      Reward_total : -90.0      Reward_local_mean : -90.6
Episode : 571      Reward_total : -91.0      Reward_local_mean : -90.5
Episode : 581      Reward_total : -92.0      Reward_local_mean : -89.6
Episode : 591      Reward_total : -92.0      Reward_local_mean : -89.9
Episode : 601      Reward_total : -89.0      Reward_local_mean : -89.9
Episode : 611      Reward_total : -85.0      Reward_local_mean : -87.1
Episode : 621      Reward_total : -91.0      Reward_local_mean : -90.3
Episod